In [ ]:
# !pip install torch torchvision torchaudio
# !pip install torch-geometric
!pip install torch torchvision torchaudio torch-geometric scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.5 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd

In [ ]:
!pip install gdown
!gdown --no-check-certificate --folder https://drive.google.com/drive/folders/17g55PHmMWFo6aBNhmzjOxVDfwtSDFUl3?usp=drive_link
# !gdown --no-check-certificate --folder https://drive.google.com/drive/folders/1Uf9L2NuLPFcnr0ydykXHnBQKE5ofKCBU?usp=drive_link

Retrieving folder contents
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'drive.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Processing file 1J2eOhACVhee6fnrO0wA5ct255sVR5EQS BCNET_regular.csv
Processing file 1d4e4g7PNVkxO-H4ZknemlWZNnWOpJkLV Code_Red_I.csv
Processing file 1IleVfZkR-EQ0X6-TmM1UXZhDphFn-auG Moscow_blackout.csv
Processing file 1YFeW7-KnN474mkULMOAs3h3cxFS7tdb4 Nimda.csv
Processing file 1_PlE4ABTghvpYQ7M1MfT8w5xQmXopQok RIPE_regular.csv
Processing file 1WjlwJW4dHABkkx0ANAfAQBLIo9OZf5C1 Slammer.csv
Processing file 1RKRYfuaEmRCsgUaHIZICxL9FngrMYP7u WannaCrypt.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unv

In [ ]:
data = pd.read_csv("BGP_RIPE_datasets/Nimda.csv")

In [ ]:
data.head()

,0,0.1,0.2,8,64,1,123,4,6,11,...,0.17,0.18,0.19,0.20,0.21,57,0.22,7,242,-1
0,1,0,1,7,47,5,160,29,6,11,...,0,0,0,0,0,24,0,23,273,-1
1,2,0,2,6,37,5,77,18,5,9,...,0,0,0,0,0,29,0,8,233,-1
2,3,0,3,2,34,5,76,21,6,11,...,0,0,0,0,0,26,0,8,252,-1
3,4,0,4,4,29,4,92,13,6,9,...,0,0,0,0,0,24,0,5,261,-1
4,5,0,5,0,37,3,138,5,5,8,...,0,0,0,0,0,26,0,11,273,-1


### Result used in report (training on Nimda and Slammer and evaluating performance on Code Red I)

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import accuracy_score, f1_score
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.nn import Linear
from torch.optim import Adam

# Load datasets
nimda_path = "/content/BGP_RIPE_datasets/Nimda.csv"
slammer_path = "/content/BGP_RIPE_datasets/Slammer.csv"
code_red_path = "/content/BGP_RIPE_datasets/Code_Red_I.csv"

nimda_df = pd.read_csv(nimda_path)
slammer_df = pd.read_csv(slammer_path)
code_red_df = pd.read_csv(code_red_path)

# Preprocess Nimda and Slammer datasets
nimda_features = nimda_df.iloc[:, :-1].values
nimda_target = nimda_df.iloc[:, -1].values

slammer_features = slammer_df.iloc[:, :-1].values
slammer_target = slammer_df.iloc[:, -1].values

# Combine Nimda and Slammer datasets
combined_features = np.vstack([nimda_features, slammer_features])
combined_target = np.concatenate([nimda_target, slammer_target])

# Normalize features
scaler = MinMaxScaler()
combined_features_scaled = scaler.fit_transform(combined_features)

# Construct a graph using KNN
k = 5  # Number of neighbors
graph = kneighbors_graph(combined_features_scaled, n_neighbors=k, mode="connectivity", include_self=False)
edge_index = torch.tensor(np.array(graph.nonzero()), dtype=torch.long)

# Create PyTorch Geometric data object
x = torch.tensor(combined_features_scaled, dtype=torch.float)
y = torch.tensor(combined_target, dtype=torch.long)

# Split data into training, validation, and test sets
num_nodes = x.shape[0]
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[:int(0.7 * num_nodes)] = True
val_mask[int(0.7 * num_nodes):] = True

data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, val_mask=val_mask)

# Define the GNN model
class GNNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = self.fc(x)
        return x

# Train the GNN model
input_dim = data.num_features
hidden_dim = 64
output_dim = len(torch.unique(data.y))
epochs = 50
learning_rate = 0.01

model = GNNModel(input_dim, hidden_dim, output_dim)
optimizer = Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(epochs):
    data.y[data.y == -1] = 0  # Map -1 to 0
    data.y[data.y == 1] = 1   # Map 1 to 1
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        out = model(data)
        _, pred = out[data.val_mask].max(dim=1)
        val_acc = (pred == data.y[data.val_mask]).sum() / data.val_mask.sum()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Val Accuracy: {val_acc:.4f}")

# Preprocess Code Red dataset for testing
code_red_features = code_red_df.iloc[:, :-1].values
code_red_target = code_red_df.iloc[:, -1].values

# Normalize features
code_red_features_scaled = scaler.transform(code_red_features)

# Convert to PyTorch tensors
test_x = torch.tensor(code_red_features_scaled, dtype=torch.float)
test_y = torch.tensor(code_red_target, dtype=torch.long)

# Test the trained model
model.eval()
with torch.no_grad():
    out = model(Data(x=test_x, edge_index=edge_index))  # Use training graph structure for simplicity
    predictions = out.argmax(dim=1)

# # Evaluate accuracy and F1 score
# accuracy = accuracy_score(test_y.numpy(), predictions.numpy())
# f1 = f1_score(test_y.numpy(), predictions.numpy(), average="weighted")

# print(f"Test Accuracy: {accuracy:.4f}")
# print(f"Test F1 Score: {f1:.4f}")
predval = predictions.numpy()
trueval = test_y.numpy()

print(classification_report(trueval, predval))


                precision      recall    f1-score   support

           0       0.9781      0.9731      0.9756      3120
           1       0.9656      0.9721      0.9689        42

    accuracy                               0.9778      3162
   macro avg       0.9718      0.9726      0.9722      3162
weighted avg       0.9777      0.9778      0.9777      3162



#### Example: Training on Nimda and Slammer and testing on Code Red I

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import kneighbors_graph
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.nn import Linear
from torch.optim import Adam

# Step 1: Load the datasets
nimda_path = "/content/BGP_RIPE_datasets/Nimda.csv"
slammer_path = "/content/BGP_RIPE_datasets/Slammer.csv"

nimda_df = pd.read_csv(nimda_path)
slammer_df = pd.read_csv(slammer_path)

# Step 2: Preprocess the data
# Separate features and target
nimda_features = nimda_df.iloc[:, :-1].values
nimda_target = nimda_df.iloc[:, -1].values

slammer_features = slammer_df.iloc[:, :-1].values
slammer_target = slammer_df.iloc[:, -1].values

# Normalize features
scaler = MinMaxScaler()
nimda_features_scaled = scaler.fit_transform(nimda_features)
slammer_features_scaled = scaler.fit_transform(slammer_features)

# Combine datasets
combined_features = np.vstack([nimda_features_scaled, slammer_features_scaled])
combined_target = np.concatenate([nimda_target, slammer_target])

# Step 3: Construct the graph using KNN
k = 5  # Number of neighbors
graph = kneighbors_graph(combined_features, n_neighbors=k, mode='connectivity', include_self=False)
edge_index = torch.tensor(np.array(graph.nonzero()), dtype=torch.long)

# Create PyTorch Geometric data object
x = torch.tensor(combined_features, dtype=torch.float)
y = torch.tensor(combined_target, dtype=torch.long)

# Define train, validation, and test masks
num_nodes = x.shape[0]
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[:int(0.6 * num_nodes)] = True
val_mask[int(0.6 * num_nodes):int(0.8 * num_nodes)] = True
test_mask[int(0.8 * num_nodes):] = True

data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)

# Step 4: Define the GNN model
class GNNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = self.fc(x)
        return x

# Step 5: Train the model
input_dim = data.num_features
hidden_dim = 64
output_dim = len(torch.unique(data.y))
epochs = 50
learning_rate = 0.01

model = GNNModel(input_dim, hidden_dim, output_dim)
optimizer = Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(epochs):
    data.y[data.y == -1] = 0  # Map -1 to 0
    data.y[data.y == 1] = 1   # Map 1 to 1 (if needed)
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    _, pred = out.max(dim=1)
    correct = (pred[data.val_mask] == data.y[data.val_mask]).sum()
    acc = int(correct) / int(data.val_mask.sum())
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Val Accuracy: {acc:.4f}")

# Step 6: Evaluate the model
model.eval()
_, pred = model(data).max(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f"Test Accuracy: {acc:.4f}")

val_pred = pred[data.test_mask].numpy()
val_true = data.y[data.test_mask].numpy()


Epoch 1, Loss: 0.7117, Val Accuracy: 0.4030
Epoch 2, Loss: 0.5831, Val Accuracy: 0.7384
Epoch 3, Loss: 0.4919, Val Accuracy: 0.7384
Epoch 4, Loss: 0.4287, Val Accuracy: 0.7384
Epoch 5, Loss: 0.4132, Val Accuracy: 0.7384
Epoch 6, Loss: 0.4409, Val Accuracy: 0.7384
Epoch 7, Loss: 0.4503, Val Accuracy: 0.7384
Epoch 8, Loss: 0.4358, Val Accuracy: 0.7384
Epoch 9, Loss: 0.4137, Val Accuracy: 0.7384
Epoch 10, Loss: 0.3996, Val Accuracy: 0.7384
Epoch 11, Loss: 0.3922, Val Accuracy: 0.7384
Epoch 12, Loss: 0.3944, Val Accuracy: 0.7384
Epoch 13, Loss: 0.3982, Val Accuracy: 0.7384
Epoch 14, Loss: 0.3992, Val Accuracy: 0.7384
Epoch 15, Loss: 0.3957, Val Accuracy: 0.7384
Epoch 16, Loss: 0.3915, Val Accuracy: 0.7384
Epoch 17, Loss: 0.3846, Val Accuracy: 0.7384
Epoch 18, Loss: 0.3809, Val Accuracy: 0.7384
Epoch 19, Loss: 0.3784, Val Accuracy: 0.7384
Epoch 20, Loss: 0.3770, Val Accuracy: 0.7384
Epoch 21, Loss: 0.3798, Val Accuracy: 0.7384
Epoch 22, Loss: 0.3774, Val Accuracy: 0.7384
Epoch 23, Loss: 0.3

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

code_red_1_path = "/content/BGP_RIPE_datasets/Code_Red_I.csv"
code_red_1_df = pd.read_csv(code_red_1_path)

code_red_1_features = code_red_1_df.iloc[:, :-1].values
code_red_1_target = code_red_1_df.iloc[:, -1].values


# Normalize features
scaler = MinMaxScaler()
code_red_1_features_scaled = scaler.fit_transform(code_red_1_features)

# Construct the graph using KNN for Code Red I
k = 5  # Number of neighbors
graph = kneighbors_graph(code_red_1_features_scaled, n_neighbors=k, mode='connectivity', include_self=False) # use scaled features
edge_index = torch.tensor(np.array(graph.nonzero()), dtype=torch.long)

# Create PyTorch Geometric data object for Code Red I
x = torch.tensor(code_red_1_features_scaled, dtype=torch.float) # use scaled features
y = torch.tensor(code_red_1_target, dtype=torch.long)

num_nodes = x.shape[0]
test_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask[int(0.0 * num_nodes):] = True

# Create a Data object for Code Red I
code_red_1_data = Data(x=x, edge_index=edge_index, y=y, test_mask=test_mask)


model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    out = model(code_red_1_data)  # Forward pass, using the Data object for Code Red I
    predictions = out.max(dim=1)[1]  # Get predicted class

# Evaluate accuracy and F1 score
accuracy = accuracy_score(y.numpy(), predictions.numpy())
f1 = f1_score(y.numpy(), predictions.numpy(), average="weighted")

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 96.7662
F1 Score: 95.2218


#### extra test (bad)

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import classification_report
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.nn import Linear
from torch.optim import Adam

# Load datasets
nimda_path = "/content/BGP_RIPE_datasets/Nimda.csv"
slammer_path = "/content/BGP_RIPE_datasets/Slammer.csv"

nimda_df = pd.read_csv(nimda_path)
slammer_df = pd.read_csv(slammer_path)

# Preprocess Nimda and Slammer datasets
nimda_features = nimda_df.iloc[:, :-1].values
nimda_target = nimda_df.iloc[:, -1].values

slammer_features = slammer_df.iloc[:, :-1].values
slammer_target = slammer_df.iloc[:, -1].values

# Combine Nimda and Slammer datasets
combined_features = np.vstack([nimda_features, slammer_features])
combined_target = np.concatenate([nimda_target, slammer_target])

# Normalize features
scaler = MinMaxScaler()
combined_features_scaled = scaler.fit_transform(combined_features)

# Construct a graph using KNN
k = 5  # Number of neighbors
graph = kneighbors_graph(combined_features_scaled, n_neighbors=k, mode="connectivity", include_self=False)
edge_index = torch.tensor(np.array(graph.nonzero()), dtype=torch.long)

# Create PyTorch Geometric data object
x = torch.tensor(combined_features_scaled, dtype=torch.float)
y = torch.tensor(combined_target, dtype=torch.long)

# Split data into training and validation sets
num_nodes = x.shape[0]
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[:int(0.8 * num_nodes)] = True
val_mask[int(0.8 * num_nodes):] = True

data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, val_mask=val_mask)

# Define the GNN model
class GNNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = self.fc(x)
        return x

# Train the GNN model
input_dim = data.num_features
hidden_dim = 64
output_dim = len(torch.unique(data.y))
epochs = 50
learning_rate = 0.01

model = GNNModel(input_dim, hidden_dim, output_dim)
optimizer = Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(epochs):
    data.y[data.y == -1] = 0  # Map -1 to 0
    data.y[data.y == 1] = 1   # Map 1 to 1 (if needed)
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_out = model(data)
        val_pred = val_out[data.val_mask].argmax(dim=1)
        val_acc = (val_pred == data.y[data.val_mask]).sum().item() / data.val_mask.sum().item()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Val Accuracy: {val_acc:.4f}")

# Generate a classification report on the validation set
model.eval()
with torch.no_grad():
    val_out = model(data)
    val_pred = val_out[data.val_mask].argmax(dim=1).numpy()
    val_true = data.y[data.val_mask].numpy()

print("\nClassification Report on Validation Set:")
print(classification_report(val_true, val_pred))


Epoch 1, Loss: 0.6467, Val Accuracy: 0.9867
Epoch 2, Loss: 0.5372, Val Accuracy: 0.9867
Epoch 3, Loss: 0.4704, Val Accuracy: 0.9867
Epoch 4, Loss: 0.4800, Val Accuracy: 0.9867
Epoch 5, Loss: 0.4899, Val Accuracy: 0.9867
Epoch 6, Loss: 0.4695, Val Accuracy: 0.9867
Epoch 7, Loss: 0.4432, Val Accuracy: 0.9867
Epoch 8, Loss: 0.4333, Val Accuracy: 0.9867
Epoch 9, Loss: 0.4312, Val Accuracy: 0.9867
Epoch 10, Loss: 0.4312, Val Accuracy: 0.9867
Epoch 11, Loss: 0.4274, Val Accuracy: 0.9867
Epoch 12, Loss: 0.4206, Val Accuracy: 0.9867
Epoch 13, Loss: 0.4131, Val Accuracy: 0.9867
Epoch 14, Loss: 0.4077, Val Accuracy: 0.9867
Epoch 15, Loss: 0.4062, Val Accuracy: 0.9867
Epoch 16, Loss: 0.4052, Val Accuracy: 0.9867
Epoch 17, Loss: 0.4065, Val Accuracy: 0.9867
Epoch 18, Loss: 0.4023, Val Accuracy: 0.9867
Epoch 19, Loss: 0.3970, Val Accuracy: 0.9867
Epoch 20, Loss: 0.3934, Val Accuracy: 0.9867
Epoch 21, Loss: 0.3926, Val Accuracy: 0.9867
Epoch 22, Loss: 0.3925, Val Accuracy: 0.9867
Epoch 23, Loss: 0.3

#### Sequential Feature Selection